### Bidirectional Stacked LSTM is implemented here. Like other code snippets the artificially created data is used for experiments instead of commonly used mnist data set.

In [ ]:
import tensorflow as tf
from util.data import *
from tensorflow.contrib import rnn 
from math import *


#### Parameters ------------------------------------------------------------------------------------------------

In [ ]:
learning_rate=0.001
batch_size=256
display_step=100
epochs=1000

Sample_number=1000
Feature_Number=1024
DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)
GRU_Layers=3
GRU_size=500 # Number of hidden units in each layer
Input_size=int(sqrt(Feature_Number))
time_step=int(sqrt(Feature_Number))

X=tf.placeholder("float",[None,time_step,Input_size])
Y=tf.placeholder("float",[None,10])
output_keep_prob=tf.placeholder("float")

Weights={'Wo':tf.Variable(tf.random_normal([2*GRU_size,10],stddev=0.01)),
        'bo':tf.Variable(tf.random_normal([10]))} # Since we have 2 lstm at the end, one for forward lstm and other one from backward lstm

#### Model Definition ----------------------------------------------------------------------------------------------------

In [ ]:
def Bidirectional_Stacked_GRU(X,Weights):
    # the original data is of form batch_size*time_step*InputSize  and to use in GRU we need to have it in the form of
    # time_step*batch_size*InputSize
    X=tf.unstack(X,time_step,1)
    cells_fw=[]
    cells_bw=[]
    for i in range(GRU_Layers):
        cell_fw=rnn.GRUCell(GRU_size)
        cell_fw=rnn.DropoutWrapper(cell_fw,output_keep_prob=output_keep_prob)
        cells_fw.append(cell_fw)
        
        cell_bw=rnn.GRUCell(GRU_size)
        cell_bw=rnn.DropoutWrapper(cell_bw,output_keep_prob=output_keep_prob)
        cells_bw.append(cell_bw)
        
    
    Outputs,_,_=rnn.stack_bidirectional_rnn(cells_fw,cells_bw,X,dtype=tf.float32)
    
    out=tf.matmul(Outputs[-1],Weights['Wo'])+Weights['bo']
    return tf.nn.softmax(out)

In [ ]:
Prediction=Bidirectional_Stacked_GRU(X,Weights)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Prediction,labels=Y))
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
True_Predictions=tf.equal(tf.argmax(Prediction,1),tf.argmax(Y,1))
Accuracy=tf.reduce_mean(tf.cast(True_Predictions,'float'))

#### Graph running -----------------------------------------------------------------------------------------------

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        
        cost_avg=0
        for j in range(Number_of_batches):
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            # Reshape Inputs into batch_size*timesteps*Input_size_for_each_Cell form
            Trx=Trx.reshape(batch_size,time_step,Input_size)
            sess.run(train_step,feed_dict={X:Trx,Y:Try,output_keep_prob:0.8})
            cost_avg+=sess.run(cost,feed_dict={X:Trx,Y:Try,output_keep_prob:0.8})/Number_of_batches
        if i%display_step==0:
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Trx,Y:Try,output_keep_prob:1.})
            Test_x=Test_x.reshape(-1,time_step,Input_size)
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y,output_keep_prob:1.})
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (i, epochs, cost_avg, Train_Accuracy, Test_Accuracy))
sess.close()
           